Data Preprocessing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix

In [2]:
data = pd.read_csv(r"C:\Users\T.S\Downloads\first inten project (1).csv")


In [3]:
data.columns


Index(['Booking_ID', 'number of adults', 'number of children',
       'number of weekend nights', 'number of week nights', 'type of meal',
       'car parking space', 'room type', 'lead time', 'market segment type',
       'repeated', 'P-C', 'P-not-C', 'average price ', 'special requests',
       'date of reservation', 'booking status'],
      dtype='object')

In [4]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36285 entries, 0 to 36284
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Booking_ID                36285 non-null  object 
 1   number of adults          36285 non-null  int64  
 2   number of children        36285 non-null  int64  
 3   number of weekend nights  36285 non-null  int64  
 4   number of week nights     36285 non-null  int64  
 5   type of meal              36285 non-null  object 
 6   car parking space         36285 non-null  int64  
 7   room type                 36285 non-null  object 
 8   lead time                 36285 non-null  int64  
 9   market segment type       36285 non-null  object 
 10  repeated                  36285 non-null  int64  
 11  P-C                       36285 non-null  int64  
 12  P-not-C                   36285 non-null  int64  
 13  average price             36285 non-null  float64
 14  specia

In [5]:
data.duplicated()


0        False
1        False
2        False
3        False
4        False
         ...  
36280    False
36281    False
36282    False
36283    False
36284    False
Length: 36285, dtype: bool

In [6]:
data.duplicated().sum()==0


True

In [7]:
data.isnull().sum()


Booking_ID                  0
number of adults            0
number of children          0
number of weekend nights    0
number of week nights       0
type of meal                0
car parking space           0
room type                   0
lead time                   0
market segment type         0
repeated                    0
P-C                         0
P-not-C                     0
average price               0
special requests            0
date of reservation         0
booking status              0
dtype: int64

2. Try one or more feature selection method to optimize model accuracy also 

In [8]:
numerical_features = data.select_dtypes(include=['int64', 'float64']).columns
categorical_features = data.select_dtypes(include=['object', 'category']).columns


In [9]:
correlation_matrix = data[numerical_features].corr()


In [10]:
import scipy.stats as stats
import numpy as np

def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = sum(confusion_matrix.sum())
    return np.sqrt(chi2 / (n * (min(confusion_matrix.shape) - 1)))

# Function to compute Cramér's V for all pairs of categorical features
def compute_cramers_v_matrix(data, categorical_features):
    cramers_v_dict = {}
    for i, col1 in enumerate(categorical_features):
        for col2 in categorical_features[i + 1:]:
            cramers_v_dict[(col1, col2)] = cramers_v(data[col1], data[col2])
    return cramers_v_dict

cramers_v_dict = compute_cramers_v_matrix(data, categorical_features)


In [11]:
from scipy.stats import pointbiserialr

# Function to compute point biserial correlation for numerical-categorical feature pairs
def compute_num_cat_correlation(data, numerical_features, categorical_features):
    num_cat_correlation = {}
    for num_col in numerical_features:
        for cat_col in categorical_features:
            correlation, _ = pointbiserialr(data[cat_col].astype('category').cat.codes, data[num_col])
            num_cat_correlation[(num_col, cat_col)] = correlation
    return num_cat_correlation

num_cat_correlation_dict = compute_num_cat_correlation(data, numerical_features, categorical_features)


In [12]:
threshold = 0.8

# For numerical features
highly_correlated_num = [(col1, col2) for col1 in numerical_features for col2 in numerical_features if col1 != col2 and abs(correlation_matrix.loc[col1, col2]) > threshold]

# For categorical features
highly_correlated_cat = [pair for pair, value in cramers_v_dict.items() if value > threshold]

# For numerical-categorical pairs
highly_correlated_num_cat = [pair for pair, value in num_cat_correlation_dict.items() if abs(value) > threshold]


In [13]:
features_to_drop = set()
for col1, col2 in highly_correlated_num + highly_correlated_cat + highly_correlated_num_cat:
    features_to_drop.add(col2)

data_reduced = data.drop(columns=features_to_drop)


In [14]:
print("Remaining features:", data_reduced.columns)

Remaining features: Index(['Booking_ID', 'number of adults', 'number of children',
       'number of weekend nights', 'number of week nights',
       'car parking space', 'lead time', 'repeated', 'P-C', 'P-not-C',
       'average price ', 'special requests'],
      dtype='object')


In [15]:
data

Booking_ID  number of adults  number of children  \
0       INN00001                 1                   1   
1       INN00002                 1                   0   
2       INN00003                 2                   1   
3       INN00004                 1                   0   
4       INN00005                 1                   0   
...          ...               ...                 ...   
36280   INN36282                 2                   0   
36281   INN36283                 2                   0   
36282   INN36284                 2                   0   
36283   INN36285                 3                   0   
36284   INN36286                 2                   0   

       number of weekend nights  number of week nights  type of meal  \
0                             2                      5   Meal Plan 1   
1                             1                      3  Not Selected   
2                             1                      3   Meal Plan 1   
3                             0                      2   Meal Plan 1   
4                             1                      2  Not Selected   
...                         ...                    ...           ...   
36280                         0                      2   Meal Plan 2   
36281                         1                      3   Meal Plan 1   
36282                         1                      3   Meal Plan 1   
36283                         0                      4   Meal Plan 1   
36284                         0                      5   Meal Plan 1   

       car parking space    room type  lead time market segment type  \
0                      0  Room_Type 1        224             Offline   
1                      0  Room_Type 1          5              Online   
2                      0  Room_Type 1          1              Online   
3                      0  Room_Type 1        211              Online   
4                      0  Room_Type 1         48              Online   
...                  ...          ...        ...                 ...   
36280                  0  Room_Type 1        346              Online   
36281                  0  Room_Type 1         34              Online   
36282                  0  Room_Type 4         83              Online   
36283                  0  Room_Type 1        121             Offline   
36284                  0  Room_Type 4         44              Online   

       repeated  P-C  P-not-C  average price   special requests  \
0             0    0        0           88.00                 0   
1             0    0        0          106.68                 1   
2             0    0        0           50.00                 0   
3             0    0        0          100.00                 1   
4             0    0        0           77.00                 0   
...         ...  ...      ...             ...               ...   
36280         0    0        0          115.00                 1   
36281         0    0        0          107.55                 1   
36282         0    0        0          105.61                 1   
36283         0    0        0           96.90                 1   
36284         0    0        0          133.44                 3   

      date of reservation booking status  
0               10/2/2015   Not_Canceled  
1               11/6/2018   Not_Canceled  
2               2/28/2018       Canceled  
3               5/20/2017       Canceled  
4               4/11/2018       Canceled  
...                   ...            ...  
36280           9/13/2018       Canceled  
36281          10/15/2017   Not_Canceled  
36282          12/26/2018   Not_Canceled  
36283            7/6/2018   Not_Canceled  
36284          10/18/2018   Not_Canceled  

[36285 rows x 17 columns]

In [16]:
data_reduced.drop(['Booking_ID'],axis=1,inplace=True)

In [17]:
from sklearn.model_selection import train_test_split
x=data_reduced
y=data['booking status']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=41)


C:\Users\T.S\AppData\Local\Temp\ipykernel_14540\30194616.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['booking status']=data['booking status'].replace({'Not_Canceled':1,'Canceled':0})


In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix
rf = RandomForestClassifier(n_estimators=100, random_state=41)
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))


              precision    recall  f1-score   support

           0       0.85      0.76      0.80      3033
           1       0.89      0.93      0.91      6039

    accuracy                           0.88      9072
   macro avg       0.87      0.85      0.86      9072
weighted avg       0.87      0.88      0.87      9072

[[2312  721]
 [ 410 5629]]
              precision    recall  f1-score   support

           0       0.85      0.76      0.80      3033
           1       0.89      0.93      0.91      6039

    accuracy                           0.88      9072
   macro avg       0.87      0.85      0.86      9072
weighted avg       0.87      0.88      0.87      9072

[[2312  721]
 [ 410 5629]]


In [19]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
param_dists= {
    'n_estimators' : randint(100,500),
    'max_depth':[None,10,20,30],
    'min_samples_split' : randint(2,11)
}

model_tunings= RandomForestClassifier()

random_search_tunings = RandomizedSearchCV(estimator=model_tunings,param_distributions=param_dists,n_iter=50, cv=5, n_jobs=-1, verbose=2)

random_search_tunings.fit(x_train,y_train)

print("Best parameters found: ", random_search_tunings.best_params_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best parameters found:  {'max_depth': None, 'min_samples_split': 4, 'n_estimators': 386}


In [21]:
import pickle
rf = RandomForestClassifier(n_estimators=386,min_samples_split=4,random_state=41)
rf.fit(x_train,y_train)
y_pred = rf.predict(x_test)
model=rf
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

